In [1]:
%pylab inline

from tqdm import tqdm_notebook as tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2021
DAY = int('19')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

Populating the interactive namespace from numpy and matplotlib


In [78]:
lines, groups = linesplit("""
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
""".strip())

In [84]:
lines, groups = linesplit(raw_input, verbose=True)

1084 lines
39 groups
>>>
--- scanner 0 ---
518,548,922
-463,418,-234
...


In [85]:
beacon_sets = []

for group in groups:
  b = []
  for line in group[1:]:
    m = vectorize(int)(re.findall(r'\-?\d+', line))
    b.append(m)
  beacon_sets.append(array(b))

In [40]:
%%time

set1 = beacon_sets[0]

for set2 in tqdm(beacon_sets[6:7]):
  # set2 = beacon_sets[1]

  res = []

  set1set = {tuple(i) for i in set1}
  
  x = zeros_like(set2)

  for signs in itertools.product([-1, 1], repeat=3):
    for perm in itertools.permutations(arange(3)):
      for i, j, sign in zip(range(3), perm, signs):
        x[:, i] = sign * set2[:, j]

      for i1 in range(len(set1)):
        for i2 in range(len(set2)):
          delta = set1[i1] - x[i2]
          y = x + delta

          c = 0
          for i in y:
            if tuple(i) in set1set:
              c += 1
          res.append(c)
  print(max(res))

<timed exec>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


12

CPU times: user 1.35 s, sys: 14.6 ms, total: 1.37 s
Wall time: 1.39 s


In [ ]:
correct_sets = beacon_sets[0:1]
unassigned_sets = beacon_sets[1:]

In [45]:
%%time

# correct_sets = beacon_sets[0:1]
# unassigned_sets = beacon_sets[1:]

while len(unassigned_sets) > 0:
  for (set1i, set1), (set2i, set2) in itertools.product(
    list(enumerate(correct_sets))[5:15], 
    enumerate(unassigned_sets)):
    best_col = 0
    best_y = None

    set1set = {tuple(i) for i in set1}

    x = zeros_like(set2)

    for signs in itertools.product([-1, 1], repeat=3):
      for perm in itertools.permutations(arange(3)):
        for i, j, sign in zip(range(3), perm, signs):
          x[:, i] = sign * set2[:, j]

        for i1 in range(len(set1)):
          for i2 in range(len(set2)):
            delta = set1[i1] - x[i2]
            y = x + delta

            c = 0
            for i in y:
              if tuple(i) in set1set:
                c += 1
            if c > best_col:
              best_col = c
              best_y = y
    
    if best_col >= 12:
      print(f'found {best_col}: {set1i} + {set2i} | {len(correct_sets)} {len(unassigned_sets)}')
      correct_sets.append(best_y)
      unassigned_sets.pop(set2i)
      break

found 12: 5 + 0 | 29 10
found 12: 5 + 5 | 30 9
found 12: 7 + 2 | 31 8


KeyboardInterrupt: 

In [50]:
%%time

# correct_sets = beacon_sets[0:1]
# unassigned_sets = beacon_sets[1:]

set1 = beacon_sets[0]
set2 = beacon_sets[6]
best_col = 0
best_y = None

set1set = {tuple(i) for i in set1}

x = zeros_like(set2)

for signs in itertools.product([-1, 1], repeat=3):
  for perm in itertools.permutations(arange(3)):
    for i, j, sign in zip(range(3), perm, signs):
      x[:, i] = sign * set2[:, j]

    a = []
    for i1 in range(len(set1)):
      for i2 in range(len(set2)):
        delta = set1[i1] - x[i2]
        a.append(tuple(delta))
    c = collections.Counter(a)
    print(c.most_common(1))
#         y = x + delta

#         c = 0
#         for i in y:
#           if tuple(i) in set1set:
#             c += 1
#         if c > best_col:
#           best_col = c
#           best_y = y

# if best_col >= 12:
#   print(f'found {best_col}: {set1i} + {set2i} | {len(correct_sets)} {len(unassigned_sets)}')
#   correct_sets.append(best_y)
#   unassigned_sets.pop(set2i)
#   break

[((1271, 96, 460), 1)]
[((1271, 86, 470), 1)]
[((66, 1301, 460), 1)]
[((66, 86, 1675), 1)]
[((390, -305, 622), 2)]
[((56, 96, 1675), 1)]
[((1271, 96, 1384), 1)]
[((1271, 86, 1374), 1)]
[((66, 1301, 1384), 1)]
[((66, 86, 169), 1)]
[((56, 1301, 1374), 1)]
[((56, 96, 169), 1)]
[((1271, 1000, 460), 1)]
[((1271, 1010, 470), 1)]
[((66, -205, 460), 1)]
[((66, 1010, 1675), 1)]
[((-683, 141, 622), 2)]
[((56, 1000, 1675), 1)]
[((1271, 1000, 1384), 1)]
[((1271, 1010, 1374), 1)]
[((66, -205, 1384), 1)]
[((66, 1010, 169), 1)]
[((56, -205, 1374), 1)]
[((56, 1000, 169), 1)]
[((-235, 96, 460), 1)]
[((-235, 86, 470), 1)]
[((970, 1301, 460), 1)]
[((970, 86, 1675), 1)]
[((980, 1301, 470), 1)]
[((980, 96, 1675), 1)]
[((-235, 96, 1384), 1)]
[((-235, 86, 1374), 1)]
[((970, 1301, 1384), 1)]
[((970, 86, 169), 1)]
[((-18, 877, -1155), 2)]
[((980, 96, 169), 1)]
[((-235, 1000, 460), 1)]
[((-235, 1010, 470), 1)]
[((970, -205, 460), 1)]
[((970, 1010, 1675), 1)]
[((980, -205, 470), 1)]
[((980, 1000, 1675), 1)]
[((-

In [53]:
# %%time

# correct_sets = beacon_sets[0:1]
# unassigned_sets = beacon_sets[1:]

while len(unassigned_sets) > 0:
  for (set1i, set1), (set2i, set2) in itertools.product(
    list(enumerate(correct_sets))[:], 
    enumerate(unassigned_sets)):
    best_col = 0
    best_y = None

    set1set = {tuple(i) for i in set1}

    x = zeros_like(set2)

    for signs in itertools.product([-1, 1], repeat=3):
      for perm in itertools.permutations(arange(3)):
        for i, j, sign in zip(range(3), perm, signs):
          x[:, i] = sign * set2[:, j]

          a = []
          for i1 in range(len(set1)):
            for i2 in range(len(set2)):
              delta = set1[i1] - x[i2]
              a.append(tuple(delta))
          delta, c = collections.Counter(a).most_common(1)[0]
          if c < 12:
            continue
          
          y = x + delta

          c = 0
          for i in y:
            if tuple(i) in set1set:
              c += 1
          if c > best_col:
            best_col = c
            best_y = y
    
    if best_col >= 12:
      print(f'found {best_col}: {set1i} + {set2i} | {len(correct_sets)} {len(unassigned_sets)}')
      correct_sets.append(best_y)
      unassigned_sets.pop(set2i)
      break

found 12: 2 + 5 | 32 7
found 12: 3 + 5 | 33 6
found 12: 4 + 2 | 34 5
found 12: 29 + 1 | 35 4
found 12: 31 + 2 | 36 3
found 12: 32 + 0 | 37 2
found 12: 32 + 0 | 38 1


In [58]:
everything = zeros((0, 3), dtype=int)
for i in correct_sets:
  everything = r_[everything, i]

In [60]:
everything.shape

(1007, 3)

In [63]:
unique(everything, axis=0).shape

(479, 3)

In [64]:
submit1(_[0])

True

In [65]:
b1 = correct_sets

In [86]:
# %%time

correct_sets = beacon_sets[0:1]
unassigned_sets = beacon_sets[1:]
locs = zeros((1, 3), dtype=int)

for set1i in range(len(beacon_sets)):
  set1 = correct_sets[set1i]
  for set2i, set2 in list(enumerate(unassigned_sets))[::-1]:
    best_col = 0
    best_y = None
    best_delta = None

    set1set = {tuple(i) for i in set1}

    x = zeros_like(set2)

    for signs in itertools.product([-1, 1], repeat=3):
      for perm in itertools.permutations(arange(3)):
        for i, j, sign in zip(range(3), perm, signs):
          x[:, i] = sign * set2[:, j]

          a = []
          for i1 in range(len(set1)):
            for i2 in range(len(set2)):
              delta = set1[i1] - x[i2]
              a.append(tuple(delta))
          delta, c = collections.Counter(a).most_common(1)[0]
          if c < 12:
            continue

          y = x + delta

          c = 0
          for i in y:
            if tuple(i) in set1set:
              c += 1
          if c > best_col:
            best_col = c
            best_y = y
            best_delta = delta

    if best_col >= 12:
      print(f'found {best_col}: {set1i} + {set2i} | {len(correct_sets)} {len(unassigned_sets)}')
      correct_sets.append(best_y)
      unassigned_sets.pop(set2i)
#       locs = r_[locs, [locs[set1i] + best_delta]]
      locs = r_[locs, [best_delta]]
      print(locs[-1])

found 12: 0 + 5 | 1 38
[  -18   141 -1155]
found 12: 1 + 28 | 2 37
[ 1145    24 -1117]
found 12: 1 + 24 | 3 36
[-1153    -7 -1203]
found 12: 1 + 19 | 4 35
[   81     7 -2326]
found 12: 2 + 18 | 5 34
[ 1145    56 -2313]
found 12: 2 + 13 | 6 33
[ 2427    87 -1070]
found 12: 2 + 12 | 7 32
[ 1250  1314 -1052]
found 12: 3 + 26 | 8 31
[-2350    13 -1042]
found 12: 4 + 20 | 9 30
[   13 -1039 -2309]
found 12: 4 + 5 | 10 29
[    7  1338 -2337]
found 12: 5 + 24 | 11 28
[ 2394   -17 -2349]
found 12: 6 + 17 | 12 27
[2348   -2  131]
found 12: 6 + 13 | 13 26
[ 3707    53 -1210]
found 12: 6 + 7 | 14 25
[ 2360 -1078 -1085]
found 12: 9 + 10 | 15 24
[-1142 -1165 -2379]
found 12: 9 + 3 | 16 23
[   -1 -2358 -2359]
found 12: 10 + 11 | 17 22
[   10  2434 -2398]
found 12: 10 + 0 | 18 21
[  132  1356 -3524]
found 12: 11 + 18 | 19 20
[ 3649    55 -2266]
found 12: 11 + 5 | 20 19
[ 2487  1333 -2369]
found 12: 11 + 1 | 21 18
[ 2453 -1116 -2255]
found 12: 13 + 5 | 22 17
[ 3572 -1163 -1032]
found 12: 14 + 14 | 23 1

In [87]:
a = []
for i, j in itertools.product(locs, repeat=2):
#   print(i - j)
  a.append(sum(abs(i - j)))
max(a)

13113

In [77]:
a = []
for i, j in itertools.product(locs, repeat=2):
#   print(i - j)
  a.append(sum(abs(i - j)))
max(a)

82744

In [88]:
submit2(_)

True